In [1]:
from google.colab import files
uploaded = files.upload()

Saving categoryFuruyaFinal.csv to categoryFuruyaFinal.csv


In [0]:
import numpy as np
import tensorflow as tf

In [3]:
import keras as ks

Using TensorFlow backend.


In [0]:
import pandas as pd
#import GPy, GPyOpt
import random
import matplotlib.pyplot as plt
import pywt
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten,Dropout,Conv2D,Conv1D,MaxPooling2D, MaxPooling1D,Reshape
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
from keras.layers.pooling import GlobalMaxPooling1D
from sklearn import svm
from sklearn import metrics
from keras.callbacks import EarlyStopping

In [0]:
df = pd.read_csv("categoryFuruyaFinal.csv",delimiter=',')

In [0]:
df

In [0]:
data = df.values

In [0]:
xData = data[:,5:727]

In [0]:
xDemo = data[0:1,5:727]

In [0]:
xDemo

In [0]:
xDemo_int = xDemo.astype(np.int64)

In [0]:
x1 = np.array([1,2,3,4,5,6])

In [0]:
xData_int = xData.astype(np.int64)

Wavelet 変換　ここから
Level1, Level3

In [0]:
(cA,cD) = pywt.wavedec(xData_int, 'haar', level=1,axis=1)

In [0]:
cA_arr = np.array(cA)
cD_arr = np.array(cD)

In [0]:
waveData=np.concatenate([cA_arr, cD_arr], axis=1)
#level1のHaar waveletのデータ

In [0]:
waveData.shape

(5832, 720)

In [0]:
xWavetrainL1 =waveData[0:3888,:] 
xWavetestL1 = waveData[3888:5832,:]

In [0]:
xWavetestL1 = waveData[3888:5832,:]

In [0]:
xWavetestL1.shape

(1944, 715)

ここからlevel3 のwavelet変換

In [0]:
(cA3,cD3,cD2,cD1) = pywt.wavedec(xData_int, 'haar', level=3,axis=1)

In [0]:
(cA4,cD4,cD43,cD42,cD41) = pywt.wavedec(xData_int, 'haar', level=4,axis=1)

In [0]:
cA4_arr= np.array(cA4)
cD4_arr= np.array(cD4)
cD43_arr= np.array(cD43)
cD42_arr= np.array(cD42)
cD41_arr= np.array(cD41)

In [0]:
cA3_arr= np.array(cA3)
cD3_arr= np.array(cD3)
cD2_arr= np.array(cD2)
cD1_arr= np.array(cD1)

In [0]:
waveDataL3=np.concatenate([cA3_arr, cD3_arr,cD2_arr,cD1_arr], axis=1)

In [0]:
waveDataL3.shape

(5832, 720)

In [0]:
waveDataL4=np.concatenate([cA4_arr, cD4_arr,cD43_arr,cD42_arr,cD41_arr], axis=1)

In [0]:
xWavetrainL3 =waveDataL3[0:3888,:] 
xWavetestL3 = waveDataL3[3888:5832,:]

In [0]:
xWavetestL3 = waveDataL3[3888:5832,:]

In [0]:
xWavetrainL4 =waveDataL4[0:3888,:] 

In [0]:
xWavetestL4 = waveDataL4[3888:5832,:]

In [0]:
Xtrain1 = data[0:3888,5:727] ##4,5月

In [0]:
Xtrain1

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 3],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [0]:
Xtrain1.shape

(3888, 720)

In [0]:
Xtest1 = data[3888:5832,5:726]

In [0]:
Xtest1.shape

(1944, 720)

In [0]:
Ytrain1 = data[0:3888,1]

In [0]:
Ytest1 = data[3888:5832,1]

In [0]:
Ytest = data[3888:5832,1]

In [0]:
Ytest

array([1, 2, 3, ..., 1, 2, 3], dtype=object)

In [0]:
Ytest.shape

(1944,)

In [0]:
ytrain_int = Ytrain1.astype(np.int64)
ytest_int = Ytest1.astype(np.int64)

In [0]:
train_X = np.reshape(Xtrain1, (-1, 720, 1))
test_X = np.reshape(Xtest1, (-1, 720, 1))
train_Xw1 = np.reshape(xWavetrainL1, (-1, 720, 1))
test_Xw1 = np.reshape(xWavetestL1, (-1, 720, 1))
train_Xw3 = np.reshape(xWavetrainL3, (-1, 720, 1))
test_Xw3 = np.reshape(xWavetestL3, (-1, 720, 1))
#train_Xw4 = np.reshape(xWavetrainL4, (-1, 720, 1))
#test_Xw4 = np.reshape(xWavetestL4, (-1, 720, 1))

In [0]:
num_class = 4

In [0]:
Ytrain1 = to_categorical(ytrain_int, num_class)
Ytest1 = to_categorical(ytest_int , num_class)

In [0]:
Ytest1

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [31]:
model = Sequential()
model.add(Conv1D(225, 4, padding='same', input_shape=(720, 1), activation='relu'))
model.add(MaxPooling1D(8, padding='same'))
model.add(Conv1D(200, 3, padding='same', activation='relu'))
model.add(Conv1D(780, 7, padding='same', activation='relu'))
#model.add(MaxPooling1D(4, padding='same'))
model.add(Conv1D(300, 7, padding='same', activation='relu'))
model.add(MaxPooling1D(3, padding='same'))
model.add(Conv1D(300, 6, padding='same', activation='relu'))


model.add(Flatten())
model.add(Dense(150,activation='relu'))
model.add(Dropout(0.244))
model.add(Dense(num_class))
model.add(Activation('softmax'))
model.summary()
model.compile(
loss='categorical_crossentropy', 
optimizer='adam', 
metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 720, 225)          1125      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 90, 225)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 90, 200)           135200    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 90, 780)           1092780   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 90, 300)           1638300   
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 30, 300)           0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 30, 300)           540300    
__________

In [0]:
model1 = Sequential()
model1.add(Conv1D(90, 8, padding='same', input_shape=(720, 1), activation='relu'))
model1.add(MaxPooling1D(3, padding='same'))
model1.add(Conv1D(90, 8, padding='same', activation='relu'))
model1.add(Conv1D(30, 8, padding='same', activation='relu'))
model1.add(MaxPooling1D(4, padding='same'))
model1.add(Conv1D(24, 6, padding='same', activation='relu'))
model1.add(MaxPooling1D(2, padding='same'))
model1.add(Conv1D(24, 8, padding='same', activation='relu'))


model1.add(Flatten())
model1.add(Dense(168,activation='relu'))
model1.add(Dropout(0.4))
model1.add(Dense(num_class))
model1.add(Activation('softmax'))
#model1.summary()
model1.compile(
loss='categorical_crossentropy', 
optimizer='adam', 
metrics=['accuracy'])

In [0]:
model2 = Sequential()
model2.add(Conv1D(90, 8, padding='same', input_shape=(720, 1), activation='relu'))
model2.add(MaxPooling1D(3, padding='same'))
model2.add(Conv1D(90, 8, padding='same', activation='relu'))
model2.add(Conv1D(30, 8, padding='same', activation='relu'))
model2.add(MaxPooling1D(4, padding='same'))
model2.add(Conv1D(24, 6, padding='same', activation='relu'))
model2.add(MaxPooling1D(2, padding='same'))
model2.add(Conv1D(24, 8, padding='same', activation='relu'))


model2.add(Flatten())
model2.add(Dense(168,activation='relu'))
model2.add(Dropout(0.4))
model2.add(Dense(num_class))
model2.add(Activation('softmax'))
#model2.summary()
model2.compile(
loss='categorical_crossentropy', 
optimizer='adam', 
metrics=['accuracy'])

In [0]:
model3 = Sequential()
model3.add(Conv1D(90, 8, padding='same', input_shape=(720, 1), activation='relu'))
model3.add(MaxPooling1D(3, padding='same'))
model3.add(Conv1D(90, 8, padding='same', activation='relu'))
model3.add(Conv1D(30, 8, padding='same', activation='relu'))
model3.add(MaxPooling1D(4, padding='same'))
model3.add(Conv1D(24, 6, padding='same', activation='relu'))
model3.add(MaxPooling1D(2, padding='same'))
model3.add(Conv1D(24, 8, padding='same', activation='relu'))


model3.add(Flatten())
model3.add(Dense(168,activation='relu'))
model3.add(Dropout(0.4))
model3.add(Dense(num_class))
model3.add(Activation('softmax'))
#model2.summary()
model3.compile(
loss='categorical_crossentropy', 
optimizer='adam', 
metrics=['accuracy'])

In [0]:
callback = Tensorboard(logdir='drive/path/to/log')

In [0]:
train_X = np.reshape(Xtrain1, (-1, 720, 1))
test_X = np.reshape(Xtest1, (-1, 720, 1))
train_Xw1 = np.reshape(xWavetrainL1, (-1, 720, 1))
test_Xw1 = np.reshape(xWavetestL1, (-1, 720, 1))
train_Xw3 = np.reshape(xWavetrainL3, (-1, 720, 1))
test_Xw3 = np.reshape(xWavetestL3, (-1, 720, 1))

In [0]:
early_stopping = EarlyStopping(patience = 0,verbose=1)

In [34]:

fit1 = model.fit(train_X, Ytrain1,
              batch_size=150,
              epochs=120,
              verbose=1,
              validation_split=0.05,
              callbacks = [early_stopping]
              ##callbacks=[callback]
                )


Train on 3693 samples, validate on 195 samples
Epoch 1/120
3693/3693 [==============================] - 9s 2ms/step - loss: 1.3222 - acc: 0.3894 - val_loss: 0.9266 - val_acc: 0.5385
Epoch 2/120
3693/3693 [==============================] - 5s 1ms/step - loss: 0.9755 - acc: 0.5066 - val_loss: 0.8982 - val_acc: 0.5590
Epoch 3/120
3693/3693 [==============================] - 5s 1ms/step - loss: 0.9424 - acc: 0.5294 - val_loss: 0.8588 - val_acc: 0.5641
Epoch 4/120
3693/3693 [==============================] - 5s 1ms/step - loss: 0.9389 - acc: 0.5348 - val_loss: 0.8612 - val_acc: 0.5795
Epoch 00004: early stopping


In [36]:
fit2 = model.fit(train_Xw1, Ytrain1,
              batch_size=150,
              epochs=120,
              verbose=1,
              validation_split=0.05,
              callbacks = [early_stopping]
              ##callbacks=[callback]
                )


Train on 3693 samples, validate on 195 samples
Epoch 1/120
3693/3693 [==============================] - 4s 1ms/step - loss: 0.9426 - acc: 0.5340 - val_loss: 0.8769 - val_acc: 0.5641
Epoch 2/120
3693/3693 [==============================] - 4s 1ms/step - loss: 0.9262 - acc: 0.5383 - val_loss: 0.9312 - val_acc: 0.5282
Epoch 00002: early stopping


In [38]:
fit3 = model.fit(train_Xw3, Ytrain1,
              batch_size=150,
              epochs=120,
              verbose=1,
              validation_split=0.05,
              callbacks = [early_stopping]
              ##callbacks=[callback]
                )

Train on 3693 samples, validate on 195 samples
Epoch 1/120
3693/3693 [==============================] - 4s 1ms/step - loss: 0.9226 - acc: 0.5426 - val_loss: 0.8880 - val_acc: 0.5795
Epoch 2/120
3693/3693 [==============================] - 4s 1ms/step - loss: 0.9081 - acc: 0.5502 - val_loss: 0.8792 - val_acc: 0.5641
Epoch 3/120
3693/3693 [==============================] - 4s 1ms/step - loss: 0.8904 - acc: 0.5686 - val_loss: 0.8948 - val_acc: 0.5590
Epoch 00003: early stopping


In [0]:
fit4 = model3.fit(train_Xw4, Ytrain1,
              batch_size=150,
              epochs=80,
              verbose=1,
              validation_split=0.045,
              callbacks = [early_stopping]
              ##callbacks=[callback]
                )

Train on 3713 samples, validate on 175 samples
Epoch 1/80
3713/3713 [==============================] - 2s 504us/step - loss: 1.1249 - acc: 0.3962 - val_loss: 0.9869 - val_acc: 0.4914
Epoch 2/80
3713/3713 [==============================] - 1s 257us/step - loss: 1.0231 - acc: 0.4705 - val_loss: 0.8918 - val_acc: 0.5657
Epoch 3/80
3713/3713 [==============================] - 1s 252us/step - loss: 0.9862 - acc: 0.4861 - val_loss: 0.8853 - val_acc: 0.5314
Epoch 4/80
3713/3713 [==============================] - 1s 249us/step - loss: 0.9523 - acc: 0.5217 - val_loss: 0.9438 - val_acc: 0.5371
Epoch 00004: early stopping


In [0]:
callback = Tensorboard(logdir='drive/path/to/log')

In [39]:
##学習回数60回
score = model.evaluate(test_X, Ytest1,verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.025508722895948
Test accuracy: 0.5221193413184994


In [40]:
##wavelet level1
score = model.evaluate(test_Xw1, Ytest1,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.9518803676950588
Test accuracy: 0.5473251028806584


In [0]:
from sklearn import metrics

In [41]:
##wavelet level3
score = model.evaluate(test_Xw3, Ytest1,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.9501500794426404
Test accuracy: 0.5416666664213801


In [0]:
score = model2.evaluate(test_Xw4, Ytest1,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.9426698213742103
Test accuracy: 0.5246913580246914


In [0]:
 y_pred = model.predict_classes(test_X, batch_size=150, verbose=1)

1944/1944 [==============================] - 0s 247us/step


In [0]:
 y_pred1 = model1.predict_classes(test_Xw1, batch_size=150, verbose=1)

1944/1944 [==============================] - 0s 211us/step


In [0]:
y_pred2 = model2.predict_classes(test_Xw3, batch_size=150, verbose=1)

1944/1944 [==============================] - 0s 172us/step


In [0]:
y_pred3 = model3.predict_classes(test_Xw4, batch_size=150, verbose=1)

1944/1944 [==============================] - 0s 184us/step


In [0]:
y_pred

array([1, 3, 1, ..., 1, 3, 3])

In [0]:
y_pred.shape

(1944,)

In [0]:
y_pred.dtype

dtype('int64')

In [0]:
y_pred1.dtype

dtype('int64')

In [0]:
y_pred2.dtype

dtype('int64')

In [0]:
y_pred3.dtype

dtype('int64')

In [0]:
##normal
print("confusion matrix:")
print(metrics.confusion_matrix(ytest_int, y_pred))

confusion matrix:
[[535  26  87]
 [188 249 211]
 [214 184 250]]


In [0]:
(535 + 249 + 250)/1944

0.5318930041152263

In [0]:
##wavelet level1
print("confusion matrix:")
print(metrics.confusion_matrix(ytest_int, y_pred1))

confusion matrix:
[[555  32  61]
 [229 169 250]
 [236 141 271]]


In [0]:
(555 + 169 + 271)/1944

0.5118312757201646

In [0]:
##bwavelet level3
print("confusion matrix:")
print(metrics.confusion_matrix(ytest_int, y_pred2))

confusion matrix:
[[549  69  30]
 [215 371  62]
 [236 276 136]]


In [0]:
(549+371+136)/1944

0.5432098765432098

In [0]:
#level4
print("confusion matrix:")
print(metrics.confusion_matrix(ytest_int, y_pred3))

confusion matrix:
[[552  11  85]
 [262 143 243]
 [265 106 277]]


In [0]:
(552+143+277)/1944

0.5

In [0]:
y_pred1 = model1.predict_classes(test_X, batch_size=168, verbose=1)

1944/1944 [==============================] - 4s 2ms/step


In [0]:
##batch100, epoch times:30
print("confusion matrix:")
print(metrics.confusion_matrix(ytest_int, y_pred1))

confusion matrix:
[[378 107 163]
 [112 316 220]
 [145 206 297]]


In [0]:
class 

